In [1]:
import numpy as np
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from dataset import SpectrogramDataset
import time
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import pickle
import os


c:\Users\lukad\.conda\envs\audioexplore\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
spectrograms = np.load('data/training.npy')
labels = np.load('data/training_labels.npy')

print('Spectrogram shape:', spectrograms.shape)
print('Labels shape:', labels.shape)

print('Spectrogram dtype:', spectrograms.dtype)

Spectrogram shape: (52890, 32, 96)
Labels shape: (52890,)
Spectrogram dtype: float32


In [3]:
TRAINING_RATIO = 0.8
VALIDATION_RATIO = 0.1
TEST_RATIO = 0.1

if TRAINING_RATIO + VALIDATION_RATIO + TEST_RATIO != 1:
    raise ValueError('Training, validation, and test ratios must sum to 1.')

train_size = int(TRAINING_RATIO * len(spectrograms))
val_size = int(VALIDATION_RATIO * len(spectrograms))
test_size = len(spectrograms) - train_size - val_size

train_spectrograms, val_spectrograms, train_labels, val_labels = train_test_split(spectrograms, labels, test_size=val_size, random_state=42)
train_spectrograms, test_spectrograms, train_labels, test_labels = train_test_split(train_spectrograms, train_labels, test_size=test_size, random_state=42)

print('Training samples:', train_spectrograms.shape[0])
print('Validation samples:', val_spectrograms.shape[0])
print('Test samples:', test_spectrograms.shape[0])

Training samples: 42312
Validation samples: 5289
Test samples: 5289


In [4]:
# Define data transforms for data augmentation
transform = transforms.Compose([
    transforms.ToTensor(),
    # Add more transforms here
])

In [5]:
train_dataset = SpectrogramDataset(train_spectrograms, train_labels, transform=transform)
val_dataset = SpectrogramDataset(val_spectrograms, val_labels, transform=transforms.ToTensor())
test_dataset = SpectrogramDataset(test_spectrograms, test_labels, transform=transforms.ToTensor())

In [6]:
CLASSES = ['Other', 'Music', 'Human voice', 'Engine sounds', 'Alarm']

In [7]:
# Load data
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=True)

train_data = []
val_data = []

train_classes = []
val_classes = []


for batch, labels in train_loader:
    train_data.append(batch.flatten().numpy())
    train_classes.append(labels.item())

for batch, labels in val_loader:
    val_data.append(batch.flatten().numpy())
    val_classes.append(labels.item())


train_data = np.array(train_data)
val_data = np.array(val_data)
val_classes = np.array(val_classes)

print("Shape data_set_train:", train_data.shape)
print("Shape data_set_val:", val_data.shape)

Shape data_set_train: (42312, 3072)
Shape data_set_val: (5289, 3072)


In [8]:
# Train data
start_train_time = time.time()

model = make_pipeline(StandardScaler(), SVC(gamma='auto'))
model.fit(train_data, train_classes)
print("Model trained")

print("Train computation time:", time.time()-start_train_time)

In [ ]:
# Training accuracy
train_predictions = model.predict(train_data)
train_accuracy = accuracy_score(train_classes, train_predictions)
print("Training Accuracy:", train_accuracy)

Training Accuracy: 0.8241913683085531


In [ ]:
# Validation accuracy
start_val_time = time.time()

validation_predictions = model.predict(val_data)
validation_accuracy = accuracy_score(val_classes, validation_predictions)

print("Validation Accuracy:", validation_accuracy)
print("Validation Computation Time:", time.time()-start_val_time)

Validation Accuracy: 0.7801810865191147
Validation Computation Time: 163.78705477714539


In [ ]:
# Computation time for predicting once
train_data_one_label = []

for batch, labels in val_loader:
    train_data_one_label.append(batch.flatten().numpy())
    break

one_predict_time_start = time.time()
model.predict(train_data_one_label)
one_prediction_time = time.time()-one_predict_time_start
print("One prediction computation time:", one_prediction_time)


One prediction computation time: 0.048697710037231445


In [ ]:
# Model size
with open(f'svm_model_{int(100/ratio)}.pickle', 'wb') as file:
    pickle.dump(model, file)
    file.flush()
    file.close()

print("SVM model size:", os.path.getsize(f'svm_model_{int(100/ratio)}.pickle'))

SVM model size: 454216189


In [ ]:
with open(f'results_{int(100/ratio)}_percent_of_data_set.txt', 'wb') as file:
    file.write(f'Training accuracy {train_accuracy}\n'.encode())
    file.write(f'Validation accuracy {validation_accuracy}\n'.encode())
    file.write(f'MB: {int(os.path.getsize(f"svm_model_{int(100/ratio)}.pickle"))/1000000}\n'.encode())
    file.write(f'Computation time seconds for one prediction: {one_prediction_time}\n'.encode())

    file.flush()
    file.close()